In [1]:
import numpy as np

import tensorflow as tf

from keras.models import Model
from keras.layers import Dense, Input, GlobalAveragePooling2D

from tensorflow.keras.preprocessing import image

from keras_efficientnets import EfficientNetB0

from PIL import Image
import os

from keras.callbacks import EarlyStopping

from keras.callbacks import LearningRateScheduler
import keras.backend as K

from sklearn.preprocessing import LabelBinarizer

import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# définition des paramètres personalisés

# personal layers for VGG16 models
n_classes = 7
n_layers = 7
n_neurons_BeforeLast = 128
f_activation = 'relu'
f_activation_lastLayer = 'softmax'

# personal parameters for compilation
optimizer = 'Adagrad'
loss = 'categorical_crossentropy'

# personnal parameters for training (fitness)
epochs = 100
batch_size = 32

# personal parameters for datagen
len_heig = 224

#numéro de l'essai
num_essai = '0_efficientNet_data_augmentation'

file_name = (str(num_essai) + '_' + str(n_classes) + '_' + str(n_layers) +
'_' + str(n_neurons_BeforeLast) + '_' + str(f_activation) +
'_' + str(f_activation_lastLayer) + '_' + str(optimizer) +
'_' + str(loss) + '_' + str(epochs) + '_' + str(batch_size) )

In [3]:
# Création du générateur d'images avec augmentation
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalisation des valeurs des pixels entre 0 et 1
    rotation_range=0,  # Rotation aléatoire jusqu'à 40 degrés
    width_shift_range=0,  # Décalage horizontal aléatoire jusqu'à 20% de la largeur de l'image
    height_shift_range=0,  # Décalage vertical aléatoire jusqu'à 20% de la hauteur de l'image
    shear_range=0,  # Cisaillement aléatoire jusqu'à 20%
    zoom_range=0,  # Zoom aléatoire entre 80% et 120% de l'échelle originale
    horizontal_flip=False,  # Retournement horizontal aléatoire
    fill_mode='nearest',  # Stratégie de remplissage des nouveaux pixels
    validation_split=0.2  # Split pour validation
)

In [4]:
# Création des générateurs de données d'entraînement et de validation
train_generator = datagen.flow_from_directory(
    '../img_prepa',
    target_size=(len_heig, len_heig),
    batch_size=batch_size,
    class_mode='categorical',  # ou 'binary'
    subset='training'
)

Found 4480 images belonging to 7 classes.


In [5]:
validation_generator = datagen.flow_from_directory(
    '../img_prepa',
    target_size=(len_heig, len_heig),
    batch_size=batch_size,
    class_mode='categorical',  # ou 'binary'
    subset='validation'
)

Found 1120 images belonging to 7 classes.


In [6]:
# Model EfficientNetB0 loading without fully connected layers

input_shape = (len_heig, len_heig, 3)  # Spécifiez la taille de l'entrée
input_tensor = Input(shape=input_shape)
base_model = EfficientNetB0(input_tensor=input_tensor, include_top=False, weights='imagenet')#, classes=1000)

In [7]:
# Freeze convolution layers to avoid training
for layer in base_model.layers:
    layer.trainable = False

In [8]:
# Add personnal layers for training
x = base_model.output

x = GlobalAveragePooling2D()(x)

for i in range(n_layers):
    x = Dense(n_neurons_BeforeLast * (n_classes-i), activation=f_activation)(x) #personal layer


predictions = Dense(n_classes, activation=f_activation_lastLayer)(x)  

In [9]:
# generate model VGG16 with personal fully connected layers
model = Model(inputs=base_model.input, outputs=predictions)

In [10]:
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [11]:
# def scheduler(epoch, lr):
#     if epoch < 5:
#         return lr
#     else:
#         return lr * K.exp(-0.1)

# lr_scheduler = LearningRateScheduler(scheduler)

In [12]:
# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

In [13]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=epochs,
    callbacks=[early_stopping]
)

Epoch 1/100
140/140 [==============================] - 30s 86ms/step - loss: 1.9384 - accuracy: 0.1779 - val_loss: 1.9220 - val_accuracy: 0.2482
Epoch 2/100
140/140 [==============================] - 11s 75ms/step - loss: 1.9008 - accuracy: 0.2969 - val_loss: 1.8760 - val_accuracy: 0.3598
Epoch 3/100
140/140 [==============================] - 10s 72ms/step - loss: 1.8330 - accuracy: 0.3960 - val_loss: 1.7832 - val_accuracy: 0.4313
Epoch 4/100
140/140 [==============================] - 10s 73ms/step - loss: 1.6904 - accuracy: 0.4685 - val_loss: 1.5934 - val_accuracy: 0.5116
Epoch 5/100
140/140 [==============================] - 10s 73ms/step - loss: 1.4498 - accuracy: 0.5395 - val_loss: 1.3413 - val_accuracy: 0.5205
Epoch 6/100
140/140 [==============================] - 10s 71ms/step - loss: 1.2036 - accuracy: 0.6056 - val_loss: 1.1290 - val_accuracy: 0.6009
Epoch 7/100
140/140 [==============================] - 10s 72ms/step - loss: 1.0074 - accuracy: 0.6598 - val_loss: 0.9536 - val_ac

In [14]:
# Model training
# model.fit(train_generator, epochs= epochs, batch_size= batch_size, validation_data=validation_generator, callbacks=[early_stopping])#, lr_scheduler

In [15]:
model.save(f"../model_saved/{num_essai}_model.h5py")

INFO:tensorflow:Assets written to: ../model_saved/0_efficientNet_data_augmentation_model.h5py\assets


INFO:tensorflow:Assets written to: ../model_saved/0_efficientNet_data_augmentation_model.h5py\assets
c:\Users\dimle\Documents\clone_repo\lego_classification\.conda\lib\site-packages\keras\engine\functional.py:1384: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
c:\Users\dimle\Documents\clone_repo\lego_classification\.conda\lib\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)
